In [1]:
import os
from pydantic import Extra
import requests
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain.prompts import PromptTemplate
# Run chain
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS # type: ignore
from langchain_ollama import OllamaEmbeddings # type: ignore
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.environ["token"] = "Bearer token|123675a6-95f6-4fb7-bc95-30095472ae3a|02de311fd83421a7fd637bf34dc8f959caa29f39888d2919e4b9640a2220224b"
token = os.environ["token"]

In [3]:
from typing import ClassVar

parser = StrOutputParser()

class LlamaLLM(LLM):
    llm_url: ClassVar[str] = 'https://api.lab45.ai/v1.1/skills/completion/query'
    
    backend:        Optional[str]   = 'gpt-35-turbo-16k'
    temp:           Optional[float] = 0.7
    top_p:          Optional[float] = 0.1
    top_k:          Optional[int]   = 40
    n_batch:        Optional[int]   = 8
    n_threads:      Optional[int]   = 4
    n_predict:      Optional[int]   = 256
    max_tokens:     Optional[int]   = 256
    repeat_last_n:  Optional[int]   = 64
    repeat_penalty: Optional[float] = 1.18

    class Config:
        extra = Extra.forbid

    @property
    def _llm_type(self) -> str:
        return "gpt-35-turbo-16k"
    
    @property
    def _get_model_default_parameters(self):
        return {
            "max_tokens": self.max_tokens,
            #"n_predict": self.n_predict,
            "top_k": self.top_k,
            "top_p": self.top_p,
            "temperature": self.temp,
            #"n_batch": self.n_batch,
            #"repeat_penalty": self.repeat_penalty,
            #"repeat_last_n": self.repeat_last_n,
        }

    def _call(
        self,
        prompt: str,
        user: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        payload = {
        "messages": [
            {
            "content": prompt,
            "role": user
            }
        ],
        "skill_parameters": {
            "model_name": "gpt-35-turbo-16k",
            "max_output_tokens": 4096,
            "temperature": 0,
            "top_k": 5
        },
        "stream_response": False
        }

        headers = {"Content-Type": "application/json","Authorization": token}

        response = requests.post(self.llm_url, json=payload, headers=headers, verify=False)

       # print("API Response:", response.json())
        response.raise_for_status()

        return response.json()  # get the response from the API

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            "llmUrl": self.llm_url,
            'model_parameters': self._get_model_default_parameters
            }

In [4]:
llm = LlamaLLM()

In [55]:
skill1="Core Java L1"
skill2="Spring Boot L2"
skill3="Hibernate L3"
skill4="Selenium L4"
title="Senior Java Full Stack Developer"
user = "system"
prompt = """As a skilled Technical Interview Panel with advanced knowledge in technology and data science, your role is to meticulously evaluate a employer job description and provide detailed job description having below points outlined.
            Use the mentioned skills & title to generate the job description, where L1 means Beginner, L2 means Intermediate, L3 means Advanced, and L4 means Expert,
            1. **Job Title and Summary**:
            - "What are the key responsibilities?"
            
            2. **Key Responsibilities**:
            - "List the main roles and responsibilities as per title and combination of skill along with proficiency"

            3. **Qualifications and Skills**:
            - "What qualifications are required?"
            - "List the essential skills needed and mention the level of expertise required."

            4. **Experience Requirements**:
            - "How many years of experience are needed?"
            - "What type of previous experience is beneficial?"

            """ + skill1 + skill2 + skill3 + skill4 + title
prompt2 =   """
            Use the mentioned skills & title to generate the job description, where L1 means Beginner, L2 means Intermediate, L3 means Advanced, and L4 means Expert,
            Job Title and Summary:
            "What are the key responsibilities?"
            Key Responsibilities:
            "List the main roles and responsibilities as per title and combination of skill along with proficiency"
            Qualifications and Skills:
            "What qualifications are required?"
            "List the essential skills needed and mention the level of expertise required."
            Experience Requirements:
            "How many years of experience are needed?"
            "What type of previous experience is beneficial?"
            """ + skill1 + skill2 + skill3 + skill4 + title

In [6]:
#Testing
#prompt = "What is the current date and time?"
result = llm._call(prompt,user)
parsed_result = result['data']['content'] # type: ignore
parser.invoke(parsed_result)


"**Job Title and Summary**:\n- Job Title: Senior Java Full Stack Developer\n- Summary: We are looking for a Senior Java Full Stack Developer to join our team and be responsible for the development and maintenance of high-quality software products. The ideal candidate should have a strong background in Core Java, Spring Boot, Hibernate, and Selenium.\n\n**Key Responsibilities**:\n- Designing, developing, and maintaining high-quality software solutions using Core Java, Spring Boot, Hibernate, and Selenium.\n- Collaborating with cross-functional teams to define, design, and ship new features.\n- Writing clean, efficient, and maintainable code.\n- Troubleshooting, debugging, and resolving technical issues.\n- Participating in code reviews and providing constructive feedback.\n- Developing and implementing automated tests to ensure product quality.\n- Mentoring and providing technical guidance to junior team members.\n\n**Qualifications and Skills**:\n- Qualifications: Bachelor's degree in 

In [7]:
prompt2 = "Summarize it into 8 to 10 sentences keeping all the key resposibilities and skills profiencies mentioned in the job description."
result2 = llm._call(prompt,user)
parsed_result2 = result2['data']['content'] # type: ignore
parser.invoke(parsed_result2)

"**Job Title and Summary**:\n- Job Title: Senior Java Full Stack Developer\n- Summary: We are seeking a highly skilled Senior Java Full Stack Developer to join our team. The ideal candidate will be responsible for developing and maintaining high-quality software solutions, utilizing their expertise in Core Java, Spring Boot, Hibernate, and Selenium.\n\n**Key Responsibilities**:\n- Designing, developing, and maintaining high-quality software solutions using Core Java, Spring Boot, Hibernate, and Selenium\n- Collaborating with cross-functional teams to define, design, and ship new features\n- Troubleshooting, debugging, and resolving software issues\n- Writing clean, efficient, and maintainable code\n- Participating in code reviews and providing constructive feedback\n- Mentoring and guiding junior developers in best practices and coding standards\n- Keeping up-to-date with the latest industry trends and technologies\n\n**Qualifications and Skills**:\n- Qualifications: Bachelor's degree 

In [57]:
# Define the API endpoint for interacting with the agents API
agents_endpoint = f"https://api.lab45.ai/v1.1/agents"  # The endpoint where agent-related operations are available

# Define the API endpoint for interacting with the agents chatAPI
agents_endpoint2 = f"https://api.lab45.ai/v1.1/agent_chat_session/query"  # The endpoint where agent-related operations are available

# Set the headers for the request, including content type, accepted response format, and authorization token
headers = {
    'Content-Type': "application/json",  # The content type is set to JSON, meaning the body will be JSON-encoded
    'Accept': "text/event-stream, application/json",  # The client expects either event-stream or JSON as the response format
    'Authorization': token  # Replace <api_key> with your actual API key for authentication
}


In [ ]:
# Define the payload (request body) for the API call to create an agent, which includes agent details and configuration
payload = {
    "allow_all_access": False,
    "dataset_id": None, 
    "name": "JD_Agent", 
    "description": "JD Enhancing Agent", 
    "instructions": "You will be given JD that you need to browse in web and get latest update on this JD and show me the differnceYou will be given JD that you need to browse in web and get latest update on this JD and show me the differnce", 
    "max_output_tokens": 256,
    "model_name": "gpt-4", 
    "temperature": 0.3,
    "tools": ["BingSearchTool"],
    "top_k": 5,
    "type": "Toolset"
    
}

response = requests.post(agents_endpoint, headers=headers, json=payload)

# Print the response from the API call to inspect the result (status code, content, etc.)
print(response.json())

#'8dea253e-4cae-467b-88a5-88df4477c79f'
#'owners': ['bf955f58-9ed4-4d5a-9b49-971fc185e5a4']
#'tenant_id': 'a919164d-8b7c-43fb-8119-f1997d45ca4f'

{'detail': {'json': {'name': ['Agent name already exists']}}, 'message': 'Validation error'}


In [51]:
strip = parsed_result.rstrip()

In [58]:
# Define the payload (request body) for the API call, which contains the conversation details and instructions for the agent
payload = {
    "conversation_id": "",  
    "messages": [  
        {
            "content": "I have a job description and I want to enhance it with the latest updates. Can you help me looking in below content and provide me the latest updates?" + prompt2,  
            "name": "Suman",
            "role": "user"
        }
    ],
    "party_id": "8dea253e-4cae-467b-88a5-88df4477c79f",  # The unique ID of the party (here it is the agent_id which is generated in agents api)
    "party_type": "Agent",
    "save_conversation": False,  
    "stream_response": False  
}


response = requests.post(agents_endpoint2, headers=headers, json=payload)

# Print the response from the API call to inspect the result (status code, content, etc.)
print(response.json())

{'data': {'name': 'JD_Agent', 'role': 'function', 'content': 'As a Senior Java Full Stack Developer, the role involves managing the software development life cycle from planning and prioritization to testing and release. This position requires a blend of expertise in system and user requirement gathering, building Java modules, and testing features before their release to ensure high quality and functionality.\n\n**Key Responsibilities:**\n1. **Designing, Implementing, and Maintaining Java Applications:** You will play a crucial role in developing high-quality Java applications that are efficient and scalable.\n2. **Mentorship:** Providing guidance and mentorship to junior developers is an essential part of the job, helping them to improve their skills and understanding of Java development.\n\n**Required Skills and Experience:**\n- **Core Java:** Profound knowledge in Core Java is essential, as it is the fundamental technology used for backend development.\n- **Spring Boot:** Experienc